In [ ]:
import os
import sys
import rioxarray as rxr
import numpy as np
import treelite
import rasterio as rio
import matplotlib

from osgeo import gdal,gdal_array,osr

import matplotlib.pyplot as plt

import panel as pn

from tqdm import tqdm

import holoviews as hv
import geoviews as gv
gv.extension('bokeh')
#gv.extension('matplotlib')

import xarray as xr

import joblib

#import shap

#save the plots for comparison
import cartopy.crs as ccrs
from bokeh.resources import INLINE

#sys.path.append('/home/jovyan/BOREAL_MD/rfexpl')
#import rfexpl

import warnings
warnings.filterwarnings("ignore")

#shap.initjs()

In [ ]:
import glob

In [ ]:
gv.__version__

In [ ]:
## Define global variables

#Circumpolar
#minlon = -179.02
#maxlon = 180
#minlat = 49.99
#maxlat = 74.95

#Eurasia
#minlon = 25
#maxlon = 179.02
#minlat = 52.01
#maxlat = 75.00


#04/17/2023
minlon = -169
maxlon = -50
minlat = 45
maxlat = 75

bbox = [minlon, minlat, maxlon, maxlat]
#fn = '/efs/STG_Tutorial_Data/BorealHeight_RF/prediction/wc2.1_2.5m_bio_1.tif'
fn = '/explore/nobackup/projects/ilab/data/worldclim/1km/bioclim_cmip6/wc2.1_2.5m_bio/wc2.1_2.5m_bio_1.tif'
tmp = rxr.open_rasterio(fn, mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat)
lons = tmp.x.values
lats = tmp.y.values
shape = (len(lats), len(lons))
print('Expected size:')
print(shape)
tmp=None

print('Lons:')
print(len(lons))
with rio.open(fn) as src:
    meta = src.meta
#print(meta)
src.close()
# color map for prediction
## 'black','#636363','#fc8d59','#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850'
clrs = ['#fee08b','#ffffbf','#d9ef8b','#91cf60','#1a9850']
cmap_p = matplotlib.colors.LinearSegmentedColormap.from_list(name="color_p", 
                                                             colors=clrs, N=16) 

# color map for trend
brbg= matplotlib.cm.get_cmap('BrBG', 256)
grey = matplotlib.cm.get_cmap('Greys', 20)
gs = grey(np.linspace(0, 1, 20))
newcolors = brbg(np.linspace(0, 1, 256))
newcolors[85:170, :] = gs[5]
cmap_t = matplotlib.colors.ListedColormap(newcolors)

# set basemap
baseMap = gv.tile_sources.EsriImagery
#baseMapRGB = gv.util.get_tile_rgb(baseMap,bbox=(minlon, minlat, maxlon, maxlat), zoom_level=1).opts(width=800, height=400, projection=ccrs.PlateCarree())

In [ ]:
# function to aggregate predictors
def build_predictors(bio_file, bnds, pf_fn=None):
    #get permafrost
    #Circumpolar
    #pf_fn = "/efs/zwwillia/BorealHeight_Data/Global_data/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_clip_global_25min.tif"
    #Eurasia
    #pf_fn = "/efs/zwwillia/BorealHeight_Data/Eurasia_data/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_clip_eurasia_25min_ext.tif"
    #NA
    #pf_fn = "/efs/STG_Tutorial_Data/BorealHeight_RF/prediction/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_clip_25min.tif"
    try:
        pf = rxr.open_rasterio(pf_fn, mask_and_scale=True).rio.clip_box(minlon, minlat, maxlon, maxlat).sel(band=1).values.ravel()
        pf = pf.reshape(-1, 1)
        print('Perma shape')
        print(pf.shape)
    except:
        print('No Permafrost Added')
    
    #get bioclim
    bios = extract_bioclim(bio_file, bnds)
    print('Bios Shape')
    print(bios.shape)
    x = np.concatenate([pf, bios], axis=1)
    mask = np.isnan(x).any(axis=1)
    
    return x, mask

# function to clip bioclim to aoi
def extract_bioclim(files, bnds):
    # function to read, clip & reshape bioclim 2.5m data
    if len(files)==19:
        sl = [rxr.open_rasterio(file, mask_and_scale=True).rio.clip_box(bnds[0], bnds[1], bnds[2], bnds[3]).values.flatten() for file in files]
        sub = np.concatenate([sl], axis=1)
        sub = np.moveaxis(sub, 0, -1)
    else:
        ds = rxr.open_rasterio(files[0], mask_and_scale=True).rio.clip_box(bnds[0], bnds[1], bnds[2], bnds[3])
        sub = np.moveaxis(ds.values, 0, -1).reshape((-1, 19))
    return sub
    
# funtion to plot result 
def map_bioclim(arr, mode="pred"):
    if mode == "pred":
        cmap = cmap_p
        clim = (0.5, 25.5)
        logz = True
    if mode == "diff":
        cmap = "BrBG"
        clim = (-5.0, 5.0)
        logz = False
    if mode == "trend":
        cmap = cmap_t
        clim = (-5.0, 5.0)
        logz = False
        
    img_opts = dict(
        width=600, 
        height=300, 
        logz=logz,
        cmap=cmap,
        colorbar=True,
        clim = clim,
        tools=["hover"], active_tools=['wheel_zoom']
        )
    gv_dataset = gv_dataset = gv.Dataset((lons, lats, arr), ['longitude', 'latitude'], mode)
    
    return gv.Image(gv_dataset).opts(**img_opts)


## Load pretrained model

In [ ]:
#rf_model = joblib.load("/efs/STG_Tutorial_Data/BorealHeight_RF/prediction/rf_gpu.joblib")
#rf_model = joblib.load("/efs/zwwillia/STG-Tutorial/BorealHeight_RF/rf_segLandcov_gpu.joblib")
#rf_model = joblib.load("/efs/zwwillia/STG-Tutorial_v1/BorealHeight_RF/rf_polygon_Global_polyDiff_gpu.joblib")
#rf_model = joblib.load("/efs/zwwillia/STG-Tutorial_v1/BorealHeight_RF/rf_polygon_gpu.joblib")

#Filtered training data experiment (09/27/22)
#rf_model = joblib.load('./rf_polygon_NA_polyDiff_zeroed270922_noPerm_gpu.joblib')
rf_model = joblib.load('/explore/nobackup/projects/ilab/projects/forest_height/boreal/rf_model/rf_NA_ATL_max_Perm.joblib')

#Original Model
#rf_model = joblib.load("/efs/STG_Tutorial_Data/BorealHeight_RF/prediction/rf_gpu.joblib")

## Apply model on bioclim current

In [ ]:
%%time
#fn = '/efs/STG_Tutorial_Data/BorealHeight_RF/prediction/wc2.1_2.5m_bio.vrt'
pf_fn = "/explore/nobackup/projects/ilab/projects/forest_height/boreal/layers/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_25min.tif"
flist = [f"/explore/nobackup/projects/ilab/data/worldclim/1km/bioclim_cmip6/wc2.1_2.5m_bio/wc2.1_2.5m_bio_{id}.tif" for id in range(1,20)]
xc, maskc = build_predictors(flist, bbox, pf_fn=pf_fn)


In [ ]:
res = rf_model.predict(xc)

In [ ]:
# fill back nodata & reshape back to 2D
res[maskc]=np.nan
predc = res.reshape(shape)

In [ ]:
# show prediction
img0 = map_bioclim(predc, 'pred')
pn.Column(img0*baseMap)

In [ ]:
#save output arrays as GeoTiffs
#blueprint is target
def save_map(arr,meta,outPut):
    #using GDAL
    nrows,ncols = np.shape(arr)
    x_res = (maxlon-minlon)/float(ncols)
    y_res = (maxlat-minlat)/float(nrows)
    geotransform = (minlon,x_res,0,maxlat,0,-y_res)
    output_raster = gdal.GetDriverByName('GTiff').Create(outPut,ncols,nrows,1,gdal.GDT_Float32)
    output_raster.SetGeoTransform(geotransform)
    srs = osr.SpatialReference()
    srs.ImportFromEPSG(4326)
    output_raster.SetProjection(srs.ExportToWkt())
    output_raster.GetRasterBand(1).WriteArray(arr)
    output_raster.FlushCache()
    print('Raster saved!')

In [ ]:
#save current map for future analysis
#save_map(predc,meta,'/efs/zwwillia/BorealHeight_Data/Outputs/North_America_Current_filtered_noPerm_270922.tif')

## Apply model on bioclim future (CMIP6)
ref: https://www.worldclim.org/data/cmip6/cmip6_clim2.5m.html

In [ ]:
#cmip_path = "/efs/STG_Tutorial_Data/BorealHeight_RF/prediction/worldclim_future"
cmip_path = "/explore/nobackup/projects/ilab/data/worldclim/1km/bioclim_cmip6"
#md = 'IPSL-CM6A-LR'
md_list = ['BCC-CSM2-MR','CanESM5','CNRM-CM6-1','IPSL-CM6A-LR','MIROC-ES2L','MIROC6','MRI-ESM2-0']
#,'IPSL-CM6A-LR'
sn = ['ssp126','ssp245','ssp370','ssp585']
#yr = '2021-2040'
yr = '2061-2080'
#yr = '2081-2100'

#### <center>Available Combinations (CMIP6;  Year 2081-2100)</center>
| | GCM  | SSP126    | SSP245   | SSP370   | SSP585   |
|---:|:-------------|:-----------|:------|:------|:------|
| 1 | BCC-CSM2-MR  | &check; | &check;   | &check;| &check;|
| 2 | CanESM5  | &check; | &check;   | &check;| &check;|
| 3 | CNRM-CM6-1  | &check; | &check;   | &check;| &check;|
| 4 | GFDL-ESM4  | &check; | &check;   |  | |
| 5 | IPSL-CM6A-LR  | &check; | &check;   | &check;| &check;|
| 6 | MIROC-ES2L  | &check; | &check;   | &check;| &check;|
| 7 | MIROC6  | &check; | &check;   | &check;| &check;|
| 8 | MRI-ESM2-0  | &check; | &check;   | &check;| &check;|

In [ ]:
preds = []
diffs = []
#label for organizing outputs
reg = 'NorthAmerica'
#outDir = '/efs/zwwillia/BorealHeight_Data/Outputs/North_America_filtered_noPerm_270922'
pf_fn = "/explore/nobackup/projects/ilab/projects/forest_height/boreal/layers/UiO_PEX_PERPROB_3.0_20171201_2000_2016_warp_25min.tif"
with tqdm(total=len(sn)) as pbar:
    for s in sn:
        for md in md_list:
            fl = os.path.join(cmip_path, f"wc2.1_2.5m_bioc_{md}_{s}_{yr}.tif")
            pbar.write(f'processing: {os.path.basename(fl)}')
            pbar.update(1)
            x, msk= build_predictors([fl], bbox, pf_fn=pf_fn)        
            #if permafrost/soils removed
            #x = x[:,2:]
            #res = treelite.gtil.predict(tl_model, x, pred_margin=True)
            res = rf_model.predict(x, predict_model='GPU')
            res[msk]=np.nan
            pred = res.reshape(shape)
            #save to file
        
            preds.append(pred)
            diff = pred-predc
            diffs.append(diff)
            #outfile_pred = os.path.join(outDir, f"{reg}_{md}_{s}_{yr}.tif")
            #outfile_diff = os.path.join(outDir, f"{reg}_{md}_{s}_{yr}_DIFF.tif")
            #save_map(pred,meta,outfile_pred)
            #save_map(diff,meta,outfile_diff)

In [ ]:
# show maps
rows = []
for i in range(0, len(sn)):
    pimg = map_bioclim(preds[i], 'pred')
    title = f"Predicted HCan {yr} {sn[i]}"
    pimg.opts(title=title)
    dimg = map_bioclim(diffs[i], 'diff')
    title = f"CMIP6 {sn[i]} vs. Current "
    dimg.opts(title=title)
    rows.append(pimg*baseMap+dimg*baseMap)

In [ ]:
rows

In [ ]:
pn.Column(rows[0], 
          rows[1],
          rows[2],
          rows[3])

# Scatter plot among scenarios

In [ ]:
# define a pair of predictions, e.g. ('ssp126', 'ssp245') or ('current', 'ssp370')
pairs = ('current', 'ssp585')


In [ ]:
pred_dict = {'current' : predc,
            'ssp126' : preds[0],
            'ssp245': preds[1],
            'ssp370': preds[1],
            'ssp585': preds[2]}

In [ ]:
p1 = pred_dict[pairs[0]].ravel()
p1 = p1[~np.isnan(p1)]
p2 = pred_dict[pairs[1]].ravel()
p2 = p2[~np.isnan(p2)]
data = np.column_stack((p1, p2))
sample_size = 2000
s = np.random.choice(data.shape[0], sample_size, replace=False)
sample = data[s, :]
pts = hv.Points(sample).opts(color='k', marker='+', size=10)
pn.Column(pts.opts(width=800, height=400, xlabel=f"{pairs[0].upper()} Predicted h-can", ylabel=f"{pairs[1].upper()} Predicted h-can"))